In [1]:
!git clone https://github.com/pratyush-1/PolarNet.git

Cloning into 'PolarNet'...
remote: Enumerating objects: 73, done.
remote: Counting objects: 100% (73/73), done.
remote: Compressing objects: 100% (60/60), done.
remote: Total 73 (delta 21), reused 60 (delta 11), pack-reused 0 (from 0)
Receiving objects: 100% (73/73), 3.34 MiB | 14.07 MiB/s, done.
Resolving deltas: 100% (21/21), done.


In [2]:
cd PolarNet/

/content/PolarNet


In [3]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
from tqdm.autonotebook import tqdm
from polar import PolarCode
from reliability_sequence import Reliability_Sequence
from utils import errors_ber, errors_bler
import matplotlib.pyplot as plt
import os

<ipython-input-3-5417a7dd173f>:5: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


In [4]:
# Set parameters
n = 5
N = 2**n
K = 16
snr = 1
batch_size = 64
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Set random seed for reproducibility
torch.manual_seed(42)

In [5]:
# GAN Architecture
class NoiseGenerator(nn.Module):
    def __init__(self, input_dim=1):
        super(NoiseGenerator, self).__init__()
        # Input: corrupted codeword
        # Output: predicted noise
        self.model = nn.Sequential(
            nn.Linear(N, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 256),
            nn.LeakyReLU(0.2),
            nn.Linear(256, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, N)
        )

    def forward(self, x):
        # x shape: [batch_size, N]
        return self.model(x)

class Discriminator(nn.Module):
    def __init__(self):
        super(Discriminator, self).__init__()
        # Input: noise (either actual AWGN noise or generator-predicted noise)
        # Output: probability of being real AWGN noise
        self.model = nn.Sequential(
            nn.Linear(N, 128),
            nn.LeakyReLU(0.2),
            nn.Linear(128, 64),
            nn.LeakyReLU(0.2),
            nn.Linear(64, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        # x shape: [batch_size, N]
        return self.model(x)

In [6]:
def create_data(num_samples, n, K, snr,rs):
    """
    Create a reproducible polar code dataset

    Parameters:
    - num_samples: Total number of samples to generate
    - batch_size: Batch size for data generation
    - n: Polar code parameter
    - K: Information bit length
    - snr: Signal-to-Noise Ratio
    - rs: Reliability Sequence

    Returns:
    - Saved .npz file with dataset
    """

    msg_bits_list = []
    bpsk_list = []
    codeword_list = []
    corrupted_codeword_list = []

    polar = PolarCode(n, K, rs = rs, use_cuda=False,hard_decision=True)
    # Loop to generate data samples
    for i in range(num_samples):
        msg_bits = (torch.rand(K) > 0.5).float()
        bpsk = 1 - 2 * msg_bits

        codeword = polar.encode(bpsk.unsqueeze(0)).squeeze(0)
        corrupted_codewords = polar.channel(codeword.unsqueeze(0), snr).squeeze(0)

        msg_bits_list.append(msg_bits.cpu().numpy())
        bpsk_list.append(bpsk.cpu().numpy())
        codeword_list.append(codeword.cpu().numpy())
        corrupted_codeword_list.append(corrupted_codewords.cpu().numpy())

        msg_bits_array = np.stack(msg_bits_list)
        bpsk_array = np.stack(bpsk_list)
        codeword_array = np.stack(codeword_list)
        corrupted_codeword_array = np.stack(corrupted_codeword_list)

    filename = f"polar_dataset_N{2**n}_K{K}_SNR{snr}.npz"
    np.savez(filename, msg_bits=msg_bits_array, corrupted_codeword=corrupted_codeword_array, bpsk=bpsk_array, codeword=codeword_array)
    print(f"Dataset saved as {filename}")


In [7]:
# Create custom dataset
class PolarGANDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.msg_bits = torch.tensor(data['msg_bits'], dtype=torch.float32)
        self.bpsk = torch.tensor(data['bpsk'], dtype=torch.float32)
        self.corrupted_codeword = torch.tensor(data['corrupted_codeword'], dtype=torch.float32)
        self.codeword = torch.tensor(data['codeword'], dtype=torch.float32)

        # Calculate the actual noise (difference between clean and corrupted)
        self.noise = self.corrupted_codeword - self.codeword

    def __len__(self):
        return len(self.msg_bits)

    def __getitem__(self, idx):
        return {
            'msg_bits': self.msg_bits[idx],
            'bpsk': self.bpsk[idx],
            'corrupted_codeword': self.corrupted_codeword[idx],
            'codeword': self.codeword[idx],
            'noise': self.noise[idx]
        }

In [8]:
# Generate AWGN noise with specific SNR
def generate_awgn(shape, snr):
    # Convert SNR from dB to linear scale
    snr_linear = 10 ** (snr / 10)

    # Calculate noise power based on signal power = 1 (BPSK)
    noise_power = 1 / snr_linear

    # Generate Gaussian noise
    noise = torch.randn(shape, device=device) * torch.sqrt(torch.tensor(noise_power))

    return noise

In [9]:
# Training function for GAN
def train_gan(generator, discriminator, dataloader, epochs=100):
    # Loss functions
    adversarial_loss = nn.BCELoss()
    l1_loss = nn.L1Loss()

    # Optimizers
    optimizer_G = optim.Adam(generator.parameters(), lr=0.0002, betas=(0.5, 0.999))
    optimizer_D = optim.Adam(discriminator.parameters(), lr=0.0002, betas=(0.5, 0.999))

    # Training history
    g_losses = []
    d_losses = []

    # Set up polar code for evaluation
    polar = PolarCode(n, K, rs=Reliability_Sequence, use_cuda=(device == 'cuda'), hard_decision=True)

    for epoch in range(epochs):
        total_g_loss = 0
        total_d_loss = 0

        for batch in tqdm(dataloader, desc=f"Epoch {epoch+1}/{epochs}"):
            # Move batch data to device
            corrupted = batch['corrupted_codeword'].to(device)
            clean = batch['codeword'].to(device)
            msg_bits = batch['msg_bits'].to(device)
            bpsk = batch['bpsk'].to(device)
            real_noise = batch['noise'].to(device)  # This is the actual AWGN noise

            batch_size = corrupted.size(0)

            # Ground truths for adversarial loss
            valid = torch.ones(batch_size, 1, device=device)
            fake = torch.zeros(batch_size, 1, device=device)

            # -----------------
            #  Train Generator
            # -----------------
            optimizer_G.zero_grad()

            # Generate predicted noise
            gen_noise = generator(corrupted)

            # Discriminator output for generated noise
            validity = discriminator(gen_noise)

            # Calculate generator loss
            # Adversarial loss - make discriminator think the noise is real
            g_adv_loss = adversarial_loss(validity, valid)

            # L1 loss for noise prediction (direct supervision)
            g_l1_loss = l1_loss(gen_noise, real_noise)

            # Combined loss - balance between making realistic noise and accurate prediction
            lambda_l1 = 0.9  # Higher weight for accuracy in noise prediction
            g_loss = (1 - lambda_l1) * g_adv_loss + lambda_l1 * g_l1_loss

            g_loss.backward()
            optimizer_G.step()

            # -----------------
            #  Train Discriminator
            # -----------------
            optimizer_D.zero_grad()

            # Real AWGN noise samples
            pred_real = discriminator(real_noise)
            loss_real = adversarial_loss(pred_real, valid)

            # Fake noise samples (no gradient computation for generator here)
            pred_fake = discriminator(gen_noise.detach())
            loss_fake = adversarial_loss(pred_fake, fake)

            # Total discriminator loss
            d_loss = 0.5 * (loss_real + loss_fake)

            d_loss.backward()
            optimizer_D.step()

            total_g_loss += g_loss.item()
            total_d_loss += d_loss.item()

        # Calculate average losses for this epoch
        avg_g_loss = total_g_loss / len(dataloader)
        avg_d_loss = total_d_loss / len(dataloader)

        g_losses.append(avg_g_loss)
        d_losses.append(avg_d_loss)

        print(f"Epoch {epoch+1}/{epochs}, G Loss: {avg_g_loss:.4f}, D Loss: {avg_d_loss:.4f}")

        # Every 10 epochs, evaluate denoising performance
        if (epoch + 1) % 10 == 0:
            evaluate_denoising(generator, polar, dataloader, epoch)

        # Save the model checkpoint
        if (epoch + 1) % 20 == 0:
            checkpoint_path = f'gan_weights/epoch_{epoch+1}'
            os.makedirs('gan_weights', exist_ok=True)
            torch.save({
                'generator': generator.state_dict(),
                'discriminator': discriminator.state_dict(),
                'optimizer_G': optimizer_G.state_dict(),
                'optimizer_D': optimizer_D.state_dict(),
            }, checkpoint_path)

    # Plot training history
    plt.figure(figsize=(10, 5))
    plt.plot(g_losses, label='Generator Loss')
    plt.plot(d_losses, label='Discriminator Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.title('GAN Training Losses')
    plt.savefig('gan_training_loss.png')
    plt.close()

    return generator, discriminator

In [10]:
# Function to evaluate denoising performance
def evaluate_denoising(generator, polar_code, dataloader, epoch, num_samples=10):
    generator.eval()

    total_ber_direct = 0
    total_bler_direct = 0
    total_ber_scd = 0
    total_bler_scd = 0
    total_ber_standard = 0
    total_bler_standard = 0

    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            if i >= num_samples:
                break

            corrupted = batch['corrupted_codeword'].to(device)
            clean = batch['codeword'].to(device)
            msg_bits = batch['msg_bits'].to(device)
            bpsk = batch['bpsk'].to(device)
            real_noise = batch['noise'].to(device)

            # First evaluate standard SC decoding without denoising
            std_llrs, std_decoded_bits = polar_code.sc_decode_new(corrupted, snr=snr)
            ber_standard = errors_ber(bpsk, std_decoded_bits.sign()).item()
            bler_standard = errors_bler(bpsk, std_decoded_bits.sign()).item()

            total_ber_standard += ber_standard
            total_bler_standard += bler_standard

            # Generate noise prediction
            predicted_noise = generator(corrupted)

            # Get denoised signal
            denoised = corrupted - predicted_noise

            # Evaluate BER/BLER of denoised signal directly (before decoding)
            ber_direct = errors_ber(clean, denoised.sign()).item()
            bler_direct = errors_bler(clean, denoised.sign()).item()

            total_ber_direct += ber_direct
            total_bler_direct += bler_direct

            # Perform SCD on denoised signal
            llrs, decoded_bits = polar_code.sc_decode_new(denoised, snr=snr)

            # Evaluate BER/BLER after SCD
            ber_scd = errors_ber(bpsk, decoded_bits.sign()).item()
            bler_scd = errors_bler(bpsk, decoded_bits.sign()).item()

            total_ber_scd += ber_scd
            total_bler_scd += bler_scd

    avg_ber_standard = total_ber_standard / num_samples
    avg_bler_standard = total_bler_standard / num_samples
    avg_ber_direct = total_ber_direct / num_samples
    avg_bler_direct = total_bler_direct / num_samples
    avg_ber_scd = total_ber_scd / num_samples
    avg_bler_scd = total_bler_scd / num_samples

    print(f"Epoch {epoch+1} Evaluation:")
    print(f"Standard SC Decoding - BER: {avg_ber_standard:.4f}, BLER: {avg_bler_standard:.4f}")
    print(f"Direct Denoising - BER: {avg_ber_direct:.4f}, BLER: {avg_bler_direct:.4f}")
    print(f"GAN+SCD - BER: {avg_ber_scd:.4f}, BLER: {avg_bler_scd:.4f}")

    # Calculate improvement
    ber_imp = (avg_ber_standard - avg_ber_scd) / avg_ber_standard * 100
    bler_imp = (avg_bler_standard - avg_bler_scd) / avg_bler_standard * 100
    print(f"Improvement - BER: {ber_imp:.2f}%, BLER: {bler_imp:.2f}%")

    generator.train()

In [11]:
# Test function to compare standard decoding vs. GAN-enhanced decoding
def test_gan_denoiser(generator, test_dataloader):
    generator.eval()

    # Set up polar code for decoding
    polar = PolarCode(n, K, rs=Reliability_Sequence, use_cuda=(device == 'cuda'), hard_decision=True)

    # Results tracking
    standard_ber_total, standard_bler_total = 0, 0
    gan_ber_total, gan_bler_total = 0, 0

    # Track noise prediction accuracy
    noise_mse_total = 0

    with torch.no_grad():
        for batch in tqdm(test_dataloader, desc="Testing"):
            corrupted = batch['corrupted_codeword'].to(device)
            bpsk = batch['bpsk'].to(device)
            real_noise = batch['noise'].to(device)

            # Standard SC decoding (without denoising)
            standard_llrs, standard_decoded_bits = polar.sc_decode_new(corrupted, snr=snr)
            standard_ber, standard_bler = calculate_ber_bler(bpsk, standard_decoded_bits.sign())
            standard_ber_total += standard_ber
            standard_bler_total += standard_bler

            # GAN-enhanced decoding
            predicted_noise = generator(corrupted)

            # Calculate MSE between real and predicted noise
            noise_mse = nn.MSELoss()(predicted_noise, real_noise).item()
            noise_mse_total += noise_mse

            denoised = corrupted - predicted_noise
            gan_llrs, gan_decoded_bits = polar.sc_decode_new(denoised, snr=snr)
            gan_ber, gan_bler = calculate_ber_bler(bpsk, gan_decoded_bits.sign())
            gan_ber_total += gan_ber
            gan_bler_total += gan_bler

    # Calculate averages
    standard_avg_ber = standard_ber_total / len(test_dataloader)
    standard_avg_bler = standard_bler_total / len(test_dataloader)
    gan_avg_ber = gan_ber_total / len(test_dataloader)
    gan_avg_bler = gan_bler_total / len(test_dataloader)
    avg_noise_mse = noise_mse_total / len(test_dataloader)

    print("\nTest Results:")
    print(f"Average Noise Prediction MSE: {avg_noise_mse:.6f}")
    print(f"Standard SC Decoding - BER: {standard_avg_ber:.4f}, BLER: {standard_avg_bler:.4f}")
    print(f"GAN-enhanced Decoding - BER: {gan_avg_ber:.4f}, BLER: {gan_avg_bler:.4f}")

    # Calculate improvement
    ber_imp = (standard_avg_ber - gan_avg_ber) / standard_avg_ber * 100
    bler_imp = (standard_avg_bler - gan_avg_bler) / standard_avg_bler * 100
    print(f"Improvement - BER: {ber_imp:.2f}%, BLER: {bler_imp:.2f}%")

    # Visualize noise distributions
    compare_noise_distributions(generator, test_dataloader)

In [12]:
# Function to visualize and compare noise distributions
def compare_noise_distributions(generator, dataloader, num_samples=5):
    generator.eval()

    plt.figure(figsize=(15, 10))

    with torch.no_grad():
        for i, batch in enumerate(dataloader):
            if i >= num_samples:
                break

            corrupted = batch['corrupted_codeword'].to(device)
            real_noise = batch['noise'][0].cpu().numpy()  # Get a single sample

            # Generate predicted noise
            predicted_noise = generator(corrupted)[0].cpu().numpy()  # Get a single sample

            # Plot distributions
            plt.subplot(num_samples, 2, i*2+1)
            plt.hist(real_noise, bins=20, alpha=0.7, color='blue', label='Real AWGN')
            plt.title(f'Sample {i+1}: Real AWGN Noise')
            plt.grid(True, linestyle='--', alpha=0.7)

            plt.subplot(num_samples, 2, i*2+2)
            plt.hist(predicted_noise, bins=20, alpha=0.7, color='red', label='Predicted')
            plt.title(f'Sample {i+1}: Predicted Noise')
            plt.grid(True, linestyle='--', alpha=0.7)

    plt.tight_layout()
    plt.savefig('noise_distribution_comparison.png')
    plt.close()

    generator.train()

In [13]:
# Utility function to calculate BER and BLER
def calculate_ber_bler(targets, predictions):
    ber = errors_ber(targets, predictions).item()
    bler = errors_bler(targets, predictions).item()
    return ber, bler

In [35]:
num_samples = 10000
create_data(num_samples,n,K,snr,rs=Reliability_Sequence)

Dataset saved as polar_dataset_N32_K16_SNR1.npz


In [36]:
data = np.load('polar_dataset_N32_K16_SNR1.npz')

In [37]:
msg_bits = data['msg_bits']
corrupted_codeword = data['corrupted_codeword']
bpsk = data['bpsk']
codeword = data['codeword']
print(f"msg_bits shape: {msg_bits.shape}")
print(f"corrupted_codeword shape: {corrupted_codeword.shape}")
print(f"bpsk shape: {bpsk.shape}")
print(f"codeword shape: {codeword.shape}")

msg_bits shape: (10000, 16)
corrupted_codeword shape: (10000, 32)
bpsk shape: (10000, 16)
codeword shape: (10000, 32)


In [38]:
# polar = PolarCode(n, K, rs = Reliability_Sequence, use_cuda=False,hard_decision=True)
# device = 'cpu'
# ber_SC_total=0
# bler_SC_total=0
# decoded_messages = []
# SC_llrs_list = []

# for bpsk_bits, corrupted_word in tqdm(zip(bpsk, corrupted_codeword),total=len(bpsk)):
#     bpsk_tensor = torch.tensor(bpsk_bits, dtype=torch.float32,device=device).unsqueeze(0)
#     corrupted_codeword_tensor = torch.tensor(corrupted_word, dtype=torch.float32,device=device).unsqueeze(0)

#     SC_llrs, decoded_SC_msg_bits = polar.sc_decode_new(corrupted_codeword_tensor, snr=snr)

#     decoded_messages.append(decoded_SC_msg_bits.squeeze(0).cpu().numpy())
#     SC_llrs_list.append(SC_llrs.squeeze(0).cpu().numpy())
#     ber_SC = errors_ber(bpsk_tensor,decoded_SC_msg_bits.sign()).item()
#     bler_SC = errors_bler(bpsk_tensor,decoded_SC_msg_bits.sign()).item()

#     ber_SC_total+=ber_SC
#     bler_SC_total+=bler_SC

# filename = f"decodedbits_N{2**n}_K{K}_SNR{snr}.npz"
# np.savez(f"decodedbits_N{2**n}_K{K}_SNR{snr}.npz", decoded_msg_bits = decoded_messages, sc_llrs = SC_llrs_list)

In [39]:
dataset = PolarGANDataset(data)

In [40]:
dataset_size = len(dataset)
train_size = int(0.9 * dataset_size)
test_size = dataset_size - train_size

train_dataset, test_dataset = torch.utils.data.random_split(dataset, [train_size, test_size])

In [41]:
train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

print(f"Train dataset size: {len(train_dataset)}")
print(f"Test dataset size: {len(test_dataset)}")

Train dataset size: 9000
Test dataset size: 1000


In [42]:
generator = NoiseGenerator().to(device)
discriminator = Discriminator().to(device)

In [43]:
generator, discriminator = train_gan(generator, discriminator, train_dataloader, epochs=100)

Epoch 1/100:   0%|          | 0/141 [00:00<?, ?it/s]

Epoch 1/100, G Loss: 0.6643, D Loss: 0.6121


Epoch 2/100:   0%|          | 0/141 [00:00<?, ?it/s]

Epoch 2/100, G Loss: 0.5966, D Loss: 0.5277


Epoch 3/100:   0%|          | 0/141 [00:00<?, ?it/s]

...

Epoch 98/100, G Loss: 0.5926, D Loss: 0.6255


Epoch 99/100:   0%|          | 0/141 [00:00<?, ?it/s]

Epoch 99/100, G Loss: 0.5928, D Loss: 0.6264


Epoch 100/100:   0%|          | 0/141 [00:00<?, ?it/s]

Epoch 100/100, G Loss: 0.5920, D Loss: 0.6233
Epoch 100 Evaluation:
Standard SC Decoding - BER: 0.1138, BLER: 0.2625
Direct Denoising - BER: 0.1564, BLER: 0.9984
GAN+SCD - BER: 0.1805, BLER: 0.4422
Improvement - BER: -58.63%, BLER: -68.45%


In [44]:
test_gan_denoiser(generator, test_dataloader)

Testing:   0%|          | 0/16 [00:00<?, ?it/s]


Test Results:
Average Noise Prediction MSE: 0.596786
Standard SC Decoding - BER: 0.1000, BLER: 0.2420
GAN-enhanced Decoding - BER: 0.3119, BLER: 0.7279
Improvement - BER: -211.89%, BLER: -200.81%
